In [1]:
!pip install pandas

In [2]:
!pip install scipy

In [3]:
import tools as tl
import pandas as pd
import numpy as np
import os
from scipy import spatial
from scipy.stats import pearsonr, spearmanr

In [4]:
os.chdir('ml-100k/')

In [5]:
u_data = tl.read_data('u.data', columns=['user_id', 'item_id', 'rating', 'timestampsep'], separator='\t')
u_data

,user_id,item_id,rating,timestampsep
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [6]:
u_item = tl.read_data('u.item', columns=['movie_id', 'movie_title', 'release_date', 'video_release_date','IMDb_URL', 'unknown', 'Action', 
                                      'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy','Film_Noir', 
                                      'Horror', 'Musical', 'Mystery', 'Romance', 'Sci_Fi', 'Thriller', 'War', 'Western'], separator='|')

In [7]:
u_item

,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Childrens,...,Fantasy,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Sistema de Recomendações usando a técnica de Filtros Colaborativos baseados no usuário

In [8]:
# lista de filmes e usuários

movies_list = list(u_item['movie_id'])
users_list = sorted(list(u_data['user_id'].unique()))

In [9]:
# Cria um dicionário com o usuário e as notas para cada filme que ele assistiu, caso não tenha assistido a nota recebe nan

ratings_dict = {}

for i in users_list:
    ratings = []
    
    user_movies = list(u_data[(u_data['user_id']==i)]['item_id'])
    
    for k in movies_list:
        if k in user_movies:
            ratings.append(float(u_data[(u_data['user_id']==i) & (u_data['item_id']==k)]['rating']))
        else:
            ratings.append(np.nan)
            
    ratings_dict[i] = ratings

In [10]:
# cria uma matriz de notas, onde as linhas representam os filmes e as colunas os usuários

ratings_pd = pd.DataFrame(ratings_dict,) # index=list(range(1,1683)))

In [11]:
ratings_pd

,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
0,5.0,4.0,NaN,NaN,4.0,4.0,NaN,NaN,NaN,4.0,...,2.0,3.0,4.0,NaN,4.0,NaN,NaN,5.0,NaN,NaN
1,3.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,4.0,...,5.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
4,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1678,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1679,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1680,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
tl.similar_cos_user('cos', 1, ratings_pd,  u_data, 5)

Usuários similares a você recomendam os seguintes filmes:
          nota
Item          
315   4.196429
301   4.161616
527   4.132231
319   4.050000
529   4.025000
523   4.021739


In [13]:
tl.similar_cos_user('pearson', 1, ratings_pd,  u_data, 5)

/home/giovane_sylvestrin/anaconda3/envs/fiep-turma3/lib/python3.9/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


Usuários similares a você recomendam os seguintes filmes:
          nota
Item          
312   4.245714
315   4.196429
301   4.161616
319   4.050000
1038  4.011111
305   3.947917


In [14]:
tl.similar_cos_user('spearman', 1, ratings_pd, u_data, 5)

/home/giovane_sylvestrin/anaconda3/envs/fiep-turma3/lib/python3.9/site-packages/scipy/stats/stats.py:4264: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


Usuários similares a você recomendam os seguintes filmes:
          nota
Item          
312   4.245714
315   4.196429
301   4.161616
528   3.978495
316   3.911765
749   3.854839


# Sistema de Recomendações usando a técnica de Filtros Colaborativos baseados nos items

In [15]:
tl.similar_cos_item('cos', 1, ratings_pd, 5)

Filmes similares recomendados:
           nota
Item          
1467  5.000000
1024  3.600000
1261  3.250000
1484  2.666667
1420  1.333333


In [16]:
tl.similar_cos_item('pearson', 1, ratings_pd, 5)

/home/giovane_sylvestrin/anaconda3/envs/fiep-turma3/lib/python3.9/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


Filmes similares recomendados:
           nota
Item          
1592  3.600000
899   3.444444
913   2.500000
1234  2.375000
910   2.000000


In [17]:
tl.similar_cos_item('spearman', 1, ratings_pd, 5)

/home/giovane_sylvestrin/anaconda3/envs/fiep-turma3/lib/python3.9/site-packages/scipy/stats/stats.py:4264: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


Filmes similares recomendados:
        nota
Item       
1495  3.600
1631  3.500
1481  3.000
1108  2.400
1234  2.375
